### Model Describer Meetup Tutorial

In this notebook, we will be doing some brief EDA of the [bicycle trip dataset](https://www.kaggle.com/pronto/cycle-share-dataset/home). This is data from the Pronto Cycle Share system which consists of 500 bikes and 54 stations located in Seattle. 

The key question for this tutorial will be whether or not there are noticeable differences in trip duration by gender and by user age. We will not be controlling for location information (a known deficit of this tutorial). 

In this tutorial, we will be covering the following:

* [Data prep](#prep)
* [Exploratory data analysis](#eda)
* [Build neural network model](#neural)
* [Model Describer Evaluation](#mdesc_regression)
* [Model Describer Sensitivity](#mdesc_sensitivity_regression)
* [Additional thoughts](#thoughts)

In [ ]:
import os
from datetime import datetime
import warnings

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import keras
import pandas as pd

from mdesc.models import (Eval, ClassifierEval, Sensitivity, ClassifierSensitivity)

In [ ]:
# initialize plotly notebook_mode
init_notebook_mode(connected=True)

# filter out warning messages
warnings.filterwarnings('ignore')

#### Data Prep <a id='prep' />

Read in data and perform basic data manipulations

In [ ]:
# read in the trip and weather data. We will not be using the station data, but there are a number of ways we could use
# this if we took advantage of the location information. 
base_path = '/Users/jasonlewris/Desktop/cycle-share-dataset/'
# base_path = r'C:\Users\jlewris\Desktop\BikeData'
trip = pd.read_csv(os.path.join(base_path, 'trip.csv'), error_bad_lines=False)
weather = pd.read_csv(os.path.join(base_path, 'weather.csv'))

In [ ]:
def convert_date(dte):
    """
    convert string date into datetime object
    
    Parameters
    ----------
    dte - string
          datetime string object
          
    Return 
    ----------
    datetime obj
        string input converted to datetime object
    """
    try:
        dte = datetime.strptime(dte, '%m/%d/%Y %H:%M')
    except ValueError:
        dte = datetime.strptime(dte, '%m/%d/%Y')
    return dte

def return_part_date(dte, part_of_date='month'):
    """
    Pull the part_of_date from input datetime object
    
    Parameters
    ----------
    dte - datetime object
          input datetime object
    
    part_of_date - str - ['month', 'day', 'year', 'hour', 'minute']
          
    Return 
    ----------
    part of date
        part of date, i.e. year, hour, etc. 
    """
    dte = convert_date(dte)
    return getattr(dte, part_of_date)

def is_weekday(dte):
    """
    return whether a dte is a weekday or not
    
    Parameters
    ----------
    dte - datetime object
          input datetime object
              
    Return 
    ----------
    binary flag
        1 if is weekday else 0 
    """
    dte = convert_date(dte)
    if dte.weekday() not in [5, 6]:
        return 1
    else:
        return 0
    

In [ ]:
# pull relevant parts of date
trip['start_day'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='day'))
trip['start_year'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='year'))
trip['start_month'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='month'))
trip['start_hour'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='hour'))
weather['start_day'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='day'))
weather['start_year'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='year'))
weather['start_month'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='month'))

# test if date is weekday or not
trip['weekday'] = trip['starttime'].apply(lambda x: is_weekday(x))

In [ ]:
# pull out just the mean values for weather
weather_sub = weather[['Mean_Temperature_F', 'MeanDew_Point_F', 'Mean_Humidity', 
                      'Mean_Visibility_Miles', 'Mean_Wind_Speed_MPH', 'Precipitation_In', 
                      'start_day', 'start_year', 'start_month']]

In [ ]:
trip = pd.merge(trip, weather_sub, on=['start_day', 'start_year', 'start_month'], how='left')

In [ ]:
# drop end location information
trip = trip.drop(['to_station_name', 'to_station_id', 'from_station_name'], axis=1)

In [ ]:
# get age of rider
trip['rider_age'] = trip['start_year'] - trip['birthyear']

#### Basic Exploratory Data Analysis <a id='eda' />

Perform basic exploratory analysis of the bicycle data

In [ ]:
numtrips_weekday = trip.groupby(['weekday', 'start_hour'])['trip_id'].nunique().reset_index(name='numTrip')

total_weekday_trips = trip.loc[trip['weekday'] == 1]['trip_id'].nunique()
total_weekend_trips = trip.loc[trip['weekday'] == 0]['trip_id'].nunique()

numtrips_weekday['percentTrips'] = numtrips_weekday.apply(lambda x: x['numTrip']/total_weekday_trips if x['weekday'] == 1 else x['numTrip']/total_weekend_trips, axis=1)

In [ ]:
numtrips_weekday.head()

In [ ]:
# get number of trips by hour of day

weekday = numtrips_weekday['weekday'] == 1
weekend = numtrips_weekday['weekday'] == 0

trace1 = go.Bar(
    x = numtrips_weekday.loc[weekday, 'start_hour'].tolist(),
    y = numtrips_weekday.loc[weekday, 'percentTrips'].tolist(), 
    name='Weekday Trips'
)

trace2 = go.Bar(
    x = numtrips_weekday.loc[weekend, 'start_hour'].tolist(),
    y = numtrips_weekday.loc[weekend, 'percentTrips'].tolist(), 
    name='Weekend Trips'
)

data = [trace1, trace2]

layout = go.Layout(barmode='group', 
                  title='Weekend vs. Weekday Trips by Hour',
                  xaxis=dict(title='Hour'), 
                  yaxis=dict(title='Percent of Trips'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
trip_dist = trip.groupby(['weekday', 'start_hour'])['tripduration'].mean().reset_index(name='tripDist')

In [ ]:
trace1 = go.Scatter(
    x = trip_dist.loc[weekday, 'start_hour'].tolist(),
    y = trip_dist.loc[weekday, 'tripDist'].tolist(), 
    name='Weekday Trips', 
    mode='lines'
)

trace2 = go.Scatter(
    x = trip_dist.loc[weekend, 'start_hour'].tolist(),
    y = trip_dist.loc[weekend, 'tripDist'].tolist(), 
    name='Weekend Trips', 
    mode='lines'
)

data = [trace1, trace2]

layout = go.Layout(
                  title='Weekend vs. Weekday Trip Duration by Hour',
                  xaxis=dict(title='Hour'), 
                  yaxis=dict(title='Trip Duration (seconds)'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
tripgender = trip.groupby('gender')['tripduration'].mean().reset_index(name='tripDist')

data = [go.Bar(
    x=tripgender['gender'].tolist(),
    y=tripgender['tripDist'].tolist()
)]

layout = go.Layout(
                  title='Average Trip Duration (seconds) by Gender',
                  xaxis=dict(title='Gender'), 
                  yaxis=dict(title='Trip Duration (seconds)'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
corr = trip[['rider_age', 'Mean_Temperature_F', 'Mean_Humidity', 
             'tripduration', 'weekday']].corr()

trace = go.Heatmap(z=corr.values.tolist(), 
                   x=corr.index.tolist(),
                   y=corr.columns.tolist())

layout = go.Layout(
                  title='Correlation Heatmap',
    )

data = [trace]

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Build Neural Network Model <a id='neural' /a>

Build out example neural network in keras to predict trip duration

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
#from sklearn.preprocessing import train_test_split

In [ ]:
trip_model = trip.drop(['stoptime', 'bikeid', 'trip_id', 
                       'starttime', 'birthyear'], axis=1)
X = trip_model.loc[:, trip_model.columns != 'tripduration']
y = trip_model.loc[:, 'tripduration'].values.reshape((286857, 1))

In [ ]:
X.fillna(method='pad', inplace=True)

In [ ]:
def map_hours(hr):
    if hr < 6:
        return 'early_morning'
    elif hr >= 6 and hr < 10:
        return 'morning_rush'
    elif hr >= 10 and hr < 15:
        return 'mid_day'
    elif hr >= 15 and hr < 19:
        return 'evening_rush'
    else:
        return 'evening'
    
X['start_hour'] = X['start_hour'].apply(lambda x: map_hours(x))
X['weekday'] = X['weekday'].replace({0: 'weekend',
                                    1: 'weekday'})

In [ ]:
X = X.drop('from_station_id', axis=1)

categories = [ 'usertype', 
              'gender', 'start_year', 
              'start_month', 'start_hour', 'weekday']

continuous = [col for col in X.columns.tolist() if col not in categories]

In [ ]:

X_cat = pd.get_dummies(X[categories], columns=categories)

X_cat_cols = X_cat.columns.tolist()
#X_cat = X_cat.values

In [ ]:
X_cat.head(1)

In [ ]:
sc = StandardScaler()

X_cont = sc.fit_transform(X[continuous])



In [ ]:
X = np.hstack((X_cat, X_cont))

feature_names = X_cat_cols + continuous

In [ ]:
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(X, y,trip, 
                                                   test_size=0.1, 
                                                   random_state=42)

In [ ]:
X_train

In [ ]:
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu', 
               kernel_initializer='normal'))
model.add(Dropout(0.1))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))

#keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
adam = Adam(lr=0.0005)

model.compile(loss='mae', optimizer=adam)
model.fit(X_train, y_train, epochs=300, verbose=1, batch_size=512, 
         validation_split=0.2)

In [ ]:
group_test['start_hour'] = group_test['start_hour'].apply(lambda x: map_hours(x))

In [ ]:
groupby_df = group_test[['gender',  'usertype', 'start_hour']]

In [ ]:
groupby_df['gender'].fillna('Other', inplace=True)

### Model Describer Evaluation <a id='mdesc_regression' />

In [ ]:
RE = Eval(prediction_fn=model.predict, target_names='transit_time', 
                feature_names=feature_names)
res = RE.fit_transform(X=X_test, y=y_test.flatten(), 
                       groupby_df=groupby_df)

In [ ]:
# reminder of what the groupby_names to choose from are
res = RE.data_set.results

In [ ]:
res.loc[(res['x_name']=='Mean_Humidity')&(res['groupByValue'] == 'Short-Term Pass Holder')]

In [ ]:
RE.data_set.results.loc[RE.data_set.results['x_name'] == 'Short-Term Pass Holder']

In [ ]:
RE.data_set.viz_now(groupby_name='usertype')

In [ ]:
X[:, 71]